In [1]:
import pandas as pd
import pyodbc

In [2]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=MATRE\\SQLEXPRESS;"
    "DATABASE=player-role-profile;"
    "Trusted_Connection=yes;"
)

df = pd.read_sql("SELECT * FROM player_role_profile", conn)
df

C:\Users\maigu\AppData\Local\Temp\ipykernel_19404\1782825487.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM player_role_profile", conn)


,player,team,season,minutes,touches_p90,touches_wide_pct,touches_half_space_pct,touches_central_pct,progressive_carries_p90,carries_into_box_p90,pressures_p90,final_third_pressures_p90,shots_p90,xg_per_shot,passes_final_third_p90,receptions_between_lines_p90
0,Antoine Semenyo,Manchester City,2024/25,73,50.547945,0.439024,0.341463,0.219512,7.39726,1.232877,18.493151,8.630137,1.232877,0.08,None,None


In [4]:
# on-ball role-defining metrics
role_cols = [
    "touches_p90",
    "touches_wide_pct",
    "touches_half_space_pct",
    "touches_central_pct",
    "progressive_carries_p90",
    "carries_into_box_p90",
    "pressures_p90",
    "final_third_pressures_p90",
    "shots_p90",
    "xg_per_shot"
]

role_df = df[role_cols]
role_df




,touches_p90,touches_wide_pct,touches_half_space_pct,touches_central_pct,progressive_carries_p90,carries_into_box_p90,pressures_p90,final_third_pressures_p90,shots_p90,xg_per_shot
0,50.547945,0.439024,0.341463,0.219512,7.39726,1.232877,18.493151,8.630137,1.232877,0.08


In [5]:
synthetic_data = [
    # player, role label (temporary), touches, wide, half, central, prog carries, box carries, pressures, final 3rd pressures, shots, xg/shot
    ["Synthetic_Wide_Winger", 45, 0.65, 0.20, 0.15, 5.2, 0.6, 12.0, 5.0, 2.1, 0.12],
    ["Synthetic_Inside_Forward", 52, 0.25, 0.45, 0.30, 4.8, 1.4, 14.2, 6.8, 3.4, 0.18],
    ["Synthetic_Direct_Runner", 38, 0.40, 0.30, 0.30, 6.9, 2.3, 11.5, 4.1, 2.9, 0.22],
    ["Synthetic_Press_Forward", 48, 0.35, 0.35, 0.30, 5.5, 1.1, 21.0, 9.5, 1.8, 0.10],
]

synthetic_df = pd.DataFrame(
    synthetic_data,
    columns=[
        "player",
        "touches_p90",
        "touches_wide_pct",
        "touches_half_space_pct",
        "touches_central_pct",
        "progressive_carries_p90",
        "carries_into_box_p90",
        "pressures_p90",
        "final_third_pressures_p90",
        "shots_p90",
        "xg_per_shot"
    ]
)


In [7]:
ml_df = pd.concat([df[synthetic_df.columns], synthetic_df], ignore_index=True)

ml_df


,player,touches_p90,touches_wide_pct,touches_half_space_pct,touches_central_pct,progressive_carries_p90,carries_into_box_p90,pressures_p90,final_third_pressures_p90,shots_p90,xg_per_shot
0,Antoine Semenyo,50.547945,0.439024,0.341463,0.219512,7.39726,1.232877,18.493151,8.630137,1.232877,0.08
1,Synthetic_Wide_Winger,45.000000,0.650000,0.200000,0.150000,5.20000,0.600000,12.000000,5.000000,2.100000,0.12
2,Synthetic_Inside_Forward,52.000000,0.250000,0.450000,0.300000,4.80000,1.400000,14.200000,6.800000,3.400000,0.18
3,Synthetic_Direct_Runner,38.000000,0.400000,0.300000,0.300000,6.90000,2.300000,11.500000,4.100000,2.900000,0.22
4,Synthetic_Press_Forward,48.000000,0.350000,0.350000,0.300000,5.50000,1.100000,21.000000,9.500000,1.800000,0.10


In [10]:
from sklearn.preprocessing import StandardScaler

features = ml_df.drop(columns=["player"])
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


In [11]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
components = pca.fit_transform(scaled_features)

pca_df = pd.DataFrame(
    components,
    columns=["PC1", "PC2"]
)

pca_df["player"] = ml_df["player"]
pca_df


,PC1,PC2,player
0,-2.220883,-0.160146,Antoine Semenyo
1,-0.677809,3.264102,Synthetic_Wide_Winger
2,1.222245,-2.227179,Synthetic_Inside_Forward
3,3.344192,0.621853,Synthetic_Direct_Runner
4,-1.667745,-1.498631,Synthetic_Press_Forward


In [12]:
loadings = pd.DataFrame(
    pca.components_.T,
    columns=["PC1", "PC2"],
    index=features.columns
)

loadings


,PC1,PC2
touches_p90,-0.310944,-0.279270
touches_wide_pct,-0.129137,0.504375
touches_half_space_pct,0.044850,-0.502030
touches_central_pct,0.221986,-0.430613
progressive_carries_p90,-0.008749,0.062225
carries_into_box_p90,0.382151,-0.155286
pressures_p90,-0.366484,-0.290504
final_third_pressures_p90,-0.381715,-0.316333
shots_p90,0.414392,-0.127881
xg_per_shot,0.487334,-0.036796
